# Permuting the Phi matrix 

In [2]:
import pandas as pd
import pickle
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('../..')
from helper import generate_all_sequences 
from paths import*

In [3]:
paths = paths()

### Define the most important params 

In [4]:
path = paths.f4_path +'NOGTHSH/'
file = 'NOGTHSH_silkroad_elarasztas_128_blocks_211_220_samples.pkl'

In [7]:
sequences = pd.read_csv(paths.local_data + '/sequences.csv',
                        dtype={'176_185': str, '201_210': str}).set_index('participants')

In [8]:
models = ['saw', 'scissors', 'silkroad', 'camino']
days = ['186_195', '211_220']


participants = [101, 102, 103, 104, 105, 107, 109, 110, 111, 115, 117, 118, 119, 
                120, 121, 122, 123, 124, 126, 127, 128, 129, 130, 131, 132]

In [12]:
m = 'saw'
p = 101
d = '186_195'
file = 'NOGTHSH_' + m + '_elarasztas_' + str(p) +'_blocks_' + d +'_samples.pkl'
            
samples = open(path + file, 'rb')
samples = pickle.load(samples)

In [13]:
samples['samples'][0]['phi']

array([[1.81727872e-02, 3.32699087e-02, 6.42793081e-02, 8.84277996e-01],
       [9.97009301e-01, 9.98667381e-04, 9.96015936e-04, 9.96016017e-04],
       [9.72195659e-02, 1.42047298e-01, 7.51672236e-01, 9.06089981e-03],
       [2.98843441e-03, 7.04764864e-01, 1.20046638e-02, 2.80242038e-01],
       [4.36651648e-02, 1.11244781e-01, 7.84085512e-01, 6.10045429e-02],
       [1.86654213e-03, 6.55800539e-01, 1.62862623e-02, 3.26046656e-01],
       [1.42145135e-03, 3.74059043e-01, 5.29715134e-01, 9.48043720e-02],
       [4.03495712e-01, 5.77251040e-01, 9.97858364e-04, 1.82553904e-02],
       [7.66718188e-02, 6.00937441e-01, 3.57481490e-02, 2.86642592e-01],
       [4.77464285e-01, 2.73165629e-01, 2.23620741e-01, 2.57493457e-02]])

In [14]:
len(samples['samples'])

801

In [15]:
sequences.loc[p,:]

176_185    0312
201_210    0231
Name: 101, dtype: object

## Permutation

In [9]:
########################################
########################################
## Todo: 
## 1. Iterate over the samples for all participants at D8 
## 2. Permute the Phi matrices for all the samples (usually 1600)
## 3. Save the samples with the new Phi matrices with a different name.

########################################
########################################
########################################


In [18]:
def get_permuted_sequence(p):  
    _from = [item for item in sequences.loc[p,:]['176_185']] ## get the D8 sequence for participant p
    _to = [item for item in sequences.loc[p,:]['201_210']] ## get the D9 sequence for participant p
    
    print (p, 'seqs: from:', _from, 'to:',_to)
    sequence_map = (dict(zip(_from, _to))) ## create a dict from the sequences: {D8:D9}
    sequence_map = dict(sorted(sequence_map.items())) ## we sort the dictionary keys (D8 seqence) ascending: y0, y1, y2, y3
    permuted_sequence = list(sequence_map.values()) ## keep only the values of the dicttionary (D9 seqence),
                                                    ## the obtained list will give the information how the columns 
                                                    ## of D8 should be ordered. 

    
    
    permuted_sequence = ['c'+item for item in permuted_sequence]
    return (permuted_sequence)

In [19]:
# for p in participants: 
#     get_permuted_sequence(p)

In [20]:
#sequences

In [21]:
def permute_phi(phi, permuted_sequence):
    
    columns = {'c0': phi[:,0],
               'c1': phi[:,1],
               'c2': phi[:,2],
               'c3': phi[:,3]
          }
    
    permuted_phi_matrix = np.array([columns[item] for item in permuted_sequence]).transpose()
    return (permuted_phi_matrix)

In [22]:
# ########################################
# # Load the data
# ########################################
def generate_permuted_samples():
    for m in models: 
        for p in participants:


            ############################    
            ## Load the samples
            file = 'NOGTHSH_' + m + '_elarasztas_' + str(p) +'_blocks_' + d +'_samples.pkl'
            samples = open(path + file, 'rb')
            samples = pickle.load(samples)

            print ('file in:', file)


            ############################
            ## Get the permuted sequence 
            permuted_sequence = get_permuted_sequence(p)


            for i in range(len(samples['samples'])): ## strangely, there is a Phi and a phi matric in the samples.
                                                     ## here I will modify both of them (one-by-one), 
                                                     ## just to make sure. 
                phi = samples['samples'][i]['phi']
                permuted_phi = permute_phi(phi, permuted_sequence)
                samples['samples'][i]['phi'] = permuted_phi

                Phi = samples['samples'][i]['Phi']
                permuted_Phi = permute_phi(Phi, permuted_sequence)
                samples['samples'][i]['Phi'] = permuted_Phi


            file_save = 'NOGTHSH_' + m + '_elarasztas_' + str(p) + '-permute'+'_blocks_' + d +'_samples.pkl'
            print ('file out:', file_save)
            with open(path + file_save, 'wb') as file:
                pickle.dump(samples, file)

In [23]:
generate_permuted_samples()

file in: NOGTHSH_saw_elarasztas_101_blocks_186_195_samples.pkl
101 seqs: from: ['0', '3', '1', '2'] to: ['0', '2', '3', '1']
file out: NOGTHSH_saw_elarasztas_101-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_saw_elarasztas_102_blocks_186_195_samples.pkl
102 seqs: from: ['0', '3', '2', '1'] to: ['0', '2', '3', '1']
file out: NOGTHSH_saw_elarasztas_102-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_saw_elarasztas_103_blocks_186_195_samples.pkl
103 seqs: from: ['1', '0', '2', '3'] to: ['1', '3', '0', '2']
file out: NOGTHSH_saw_elarasztas_103-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_saw_elarasztas_104_blocks_186_195_samples.pkl
104 seqs: from: ['1', '0', '3', '2'] to: ['1', '3', '2', '0']
file out: NOGTHSH_saw_elarasztas_104-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_saw_elarasztas_105_blocks_186_195_samples.pkl
105 seqs: from: ['1', '2', '0', '3'] to: ['1', '2', '3', '0']
file out: NOGTHSH_saw_elarasztas_105-permute_blocks_186_195_samples.pkl
file in: NOGTHS

file in: NOGTHSH_scissors_elarasztas_123_blocks_186_195_samples.pkl
123 seqs: from: ['0', '2', '1', '3'] to: ['0', '1', '3', '2']
file out: NOGTHSH_scissors_elarasztas_123-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_scissors_elarasztas_124_blocks_186_195_samples.pkl
124 seqs: from: ['0', '2', '3', '1'] to: ['0', '3', '1', '2']
file out: NOGTHSH_scissors_elarasztas_124-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_scissors_elarasztas_126_blocks_186_195_samples.pkl
126 seqs: from: ['0', '3', '2', '1'] to: ['0', '3', '1', '2']
file out: NOGTHSH_scissors_elarasztas_126-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_scissors_elarasztas_127_blocks_186_195_samples.pkl
127 seqs: from: ['1', '0', '2', '3'] to: ['1', '3', '0', '2']
file out: NOGTHSH_scissors_elarasztas_127-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_scissors_elarasztas_128_blocks_186_195_samples.pkl
128 seqs: from: ['1', '0', '3', '2'] to: ['1', '3', '2', '0']
file out: NOGTHSH_scissors_elarasztas_128-

file in: NOGTHSH_camino_elarasztas_109_blocks_186_195_samples.pkl
109 seqs: from: ['2', '0', '1', '3'] to: ['2', '1', '0', '3']
file out: NOGTHSH_camino_elarasztas_109-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_camino_elarasztas_110_blocks_186_195_samples.pkl
110 seqs: from: ['2', '0', '3', '1'] to: ['3', '2', '1', '0']
file out: NOGTHSH_camino_elarasztas_110-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_camino_elarasztas_111_blocks_186_195_samples.pkl
111 seqs: from: ['2', '1', '0', '3'] to: ['2', '1', '3', '0']
file out: NOGTHSH_camino_elarasztas_111-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_camino_elarasztas_115_blocks_186_195_samples.pkl
115 seqs: from: ['3', '0', '1', '2'] to: ['3', '0', '2', '1']
file out: NOGTHSH_camino_elarasztas_115-permute_blocks_186_195_samples.pkl
file in: NOGTHSH_camino_elarasztas_117_blocks_186_195_samples.pkl
117 seqs: from: ['3', '1', '0', '2'] to: ['3', '2', '1', '0']
file out: NOGTHSH_camino_elarasztas_117-permute_blocks_186_1

In [24]:
sequences

,176_185,201_210
participants,,
101,0312,0231
102,0321,0231
103,1023,1302
104,1032,1320
105,1203,1230
107,1302,1023
109,2013,2103
110,2031,3210
111,2103,2130


In [25]:
################################################
################################################
###### Short demo of sequence permutation ######
################################################
################################################
permuted_sequence = get_permuted_sequence(101)

print ('permuted_sequence', permuted_sequence)

Phi = np.array([[0.1, 1.1, 2.1, 3.1],
                [0.2, 1.2, 2.2, 3.2],
                [0.3, 1.3, 2.3, 3.3],
                [0.4, 1.4, 2.4, 3.4],
                [0.5, 1.5, 2.5, 3.5],
                [0.6, 1.6, 2.6, 3.6],
                [0.7 ,1.7, 2.7, 3.7]])

columns = {'c0': Phi[:,0],
           'c1': Phi[:,1],
           'c2': Phi[:,2],
           'c3': Phi[:,3]
          }


inverted_matrix = np.array([columns[item] for item in permuted_sequence]).transpose()
inverted_matrix

101 seqs: from: ['0', '3', '1', '2'] to: ['0', '2', '3', '1']
permuted_sequence ['c0', 'c3', 'c1', 'c2']


array([[0.1, 3.1, 1.1, 2.1],
       [0.2, 3.2, 1.2, 2.2],
       [0.3, 3.3, 1.3, 2.3],
       [0.4, 3.4, 1.4, 2.4],
       [0.5, 3.5, 1.5, 2.5],
       [0.6, 3.6, 1.6, 2.6],
       [0.7, 3.7, 1.7, 2.7]])

## Check out the inverted phi matrices - whether they are really inverted

In [32]:
########################################################
########################################################
### Sanity check for the permuted Phis
########################################################
########################################################

old_phi_dict = {}
new_phi_dict = {}




for p in [115]:
#for p in participants[:5]:

    ############################    
    ## Load the samples
    file_old = 'NOGTHSH_' + m + '_elarasztas_' + str(p) +'_blocks_' + d +'_samples.pkl'
    print ('file_old:',file_old)
#     samples = open(path + file_old, 'rb')
#     samples = pickle.load(samples)
    
    ############################
    ##
    file_new = 'NOGTHSH_' + m + '_elarasztas_' + str(p) + '-permute' +'_blocks_' + d +'_samples.pkl'
    print ('file_new:',file_new)
#     permuted_samples = open(path + file_new, 'rb')
#     permuted_samples = pickle.load(permuted_samples)
    
#     ############################
#     old_phi_shapes = [item['Phi'].shape for item in samples['samples']]
#     new_phi_shapes = [item['Phi'].shape for item in permuted_samples['samples']]
#     print ('old shape equals new shape', old_phi_shapes == new_phi_shapes)
    
    
    old_phi = [item['Phi'] for item in samples['samples']]
#    new_phi = [item['Phi'] for item in permuted_samples['samples']]

    
    old_phi_dict[p] = old_phi
#    new_phi_dict[p] = new_phi


    
#     new_old_equals = [np.array_equal(new, old) for new in new_phi for old in old_phi]
#     new_old_equals_TRUE = [item for item in new_old_equals if item == True]
#     print (new_old_equals_TRUE)
    
    #print (old_phi_shapes, new_phi_shapes)

file_old: NOGTHSH_saw_elarasztas_115_blocks_186_195_samples.pkl
file_new: NOGTHSH_saw_elarasztas_115-permute_blocks_186_195_samples.pkl


In [30]:
#print(get_permuted_sequence(101))
print(get_permuted_sequence(102))

102 seqs: from: ['0', '3', '2', '1'] to: ['0', '2', '3', '1']
['c0', 'c1', 'c3', 'c2']


In [32]:
iter_n = 800
p = 102
print(old_phi_dict[p][iter_n], '\n\n',new_phi_dict[p][iter_n])

[[0.01339411 0.03001785 0.94840145 0.00818659]
 [0.01794701 0.98006095 0.00099602 0.00099602]
 [0.00099632 0.17395212 0.09244461 0.73260696]
 [0.10554746 0.39689394 0.16238942 0.33516919]
 [0.99047367 0.00105231 0.00108793 0.00738609]
 [0.58345079 0.00137963 0.16703901 0.24813057]
 [0.72242206 0.05005375 0.21298518 0.01453901]
 [0.14575591 0.31895694 0.22275621 0.31253094]
 [0.72864164 0.26491785 0.00381733 0.00262318]
 [0.10060936 0.24563455 0.47660233 0.17715377]
 [0.03928144 0.00904164 0.46584549 0.48583143]
 [0.04312712 0.11437008 0.46598392 0.37651888]] 

 [[0.01339411 0.03001785 0.00818659 0.94840145]
 [0.01794701 0.98006095 0.00099602 0.00099602]
 [0.00099632 0.17395212 0.73260696 0.09244461]
 [0.10554746 0.39689394 0.33516919 0.16238942]
 [0.99047367 0.00105231 0.00738609 0.00108793]
 [0.58345079 0.00137963 0.24813057 0.16703901]
 [0.72242206 0.05005375 0.01453901 0.21298518]
 [0.14575591 0.31895694 0.31253094 0.22275621]
 [0.72864164 0.26491785 0.00262318 0.00381733]
 [0.10060

## Generate test data named as "p-arbitrary.pkl"

In [9]:
#test_data_path = '/Users/szekelyanna/f4/cogtom/cogtom_code_bt/Data/'

In [1]:
# days = ['201_225']
# for p in participants:
#     for d in days:

#         file_in = 'elarasztas_' + str(p)+'_blocks_' + d +'.pkl'
#         print ('file in:', file_in)
#         samples = open(test_data_path + file_in, 'rb')
#         samples = pickle.load(samples)


#         file_out = 'elarasztas_' + str(p)+'-permute' +'_blocks_' + d +'.pkl'
#         print ('file out:', file_out)
#         with open(test_data_path + file_out, 'wb') as file:
#             pickle.dump(samples, file)